In [8]:
import numpy as np
import visa
import matplotlib.pyplot as plt
import time
from lantz import MessageBasedDriver, Q_
from lantz.core import Feat
from lantz.core import mfeats
    
class Generador(MessageBasedDriver):
#    def __init__(self,ID):
#        self.obj_visa=rm.open_resource(ID)
#        self.ID = ID

    # Feats punciona como un property, pero ademas acepta otras opciones
    @Feat()
    def idn(self):
        return self.query('*IDN?')

    # La idea es no usar sets and gets como metodos, sino definir propiedades
    def setFrequency(self,freq):
        self.write("SOURce1:FREQuency:FIXed "+str(freq))
        
  #  def setAmplitude(self,amp):
  #      self.write("SOURce1:VOLTage:LEVel:IMMediate:AMPLitude "+str(amp))
    # Una manera comoda de unas Feats es con mfeats, donde directametne pasas el getter, el setter, las unidades y un limite para los valroes
    amplitude = mfeats.QuantityFeat('SOURce1:VOLT:LEV:IMM:AMPL?','SOURce1:VOLT:LEV:IMM:AMPL {}',units='V',limits=(0.2,5))
        
    def setWaveform(self,waveform ='Senoidal'):
        switcher = {'Senoidal':"SIN",'Cuadrada':"SQU",'Pulso':"PULS"}
        self.write("SOURce1:FUNCtion "+switcher.get(waveform,'Senoidal'))  

        
class Osciloscopio(MessageBasedDriver):
#    def __init__(self,ID):
#        self.parameters = None
    
    set_query = MessageBasedDriver.write
    
    @Feat()
    def idn(self):
        return self.query('*IDN?')

    # Aca definimos el getter y el setter con feats. Al decirle que es en segundos, las salida es un objeto de Pint que tiene magnitu y unidades
    @Feat(units='s')
    def timebase(self):
        return self.query('HORizontal:MAIN:SCALe?')
       # self.obj_visa.write('HORizontal:DELay:SCALe {}'.format(scale))

    @timebase.setter
    def timebase(self,scale):
        return self.write('HORizontal:MAIN:SCALe '+str(scale))
       # self.obj_visa.write('HORizontal:DELay:SCALe {}'.format(scale))

    vertical_scale = mfeats.QuantityFeat('CH1:SCA?','CH1:SCA {}',units='V')

    def vpp(self):
        osci.write('MEASUrement:IMMed:TYPE PK2pk')
        return float(osci.query('MEASUREMENT:IMMed:VALue?'))
        
    def capturaPantalla(self):
        if self.parameters is None:
            YOFF_in_dl = float(self.query("WFMP:YOFF?"))
            YZERO_in_YUNits = float(self.query("WFMP:YZERO?"))
            YMUlt = float(self.query("WFMP:YMULT?"))
            self.parameters = (YOFF_in_dl,YZERO_in_YUNits,YMUlt)
        (YOFF_in_dl , YZERO_in_YUNits , YMUlt) = self.parameters
        curve_in_dl = np.array(self.query_binary_values('CURV?', datatype='b', is_big_endian=True))
        valores = ((curve_in_dl - YOFF_in_dl)*YMUlt)+YZERO_in_YUNits
        intervalo = float(osci.query('WFMPre:XINcr?'))
        tiempos = np.arange(len(valores))*intervalo
        return tiempos, valores

In [9]:
def sweepe(generador, osciloscopio, init_freq = 100, end_freq = 10100, cant_med = 100):
    paso = np.floor(np.divide(end_freq - init_freq,cant_med))
    values = np.zeros(cant_med)
    freqs = np.add(np.multiply(np.arange(cant_med), paso), init_freq)
    magni = generador.amplitude.magnitude
    valor = np.ceil(2.0 * magni / 8.0)
    print(valor)
    osciloscopio.vertical_scale = Q_(valor,'V')
    
    for i,freq in enumerate(freqs):
        periodo = np.divide(1,freq)
        # Como en el Feats definimos las unidades, al setearlo debemos escribirlo con que unidades le estamso mandando
        osciloscopio.timebase = Q_(0.1 * periodo * 4, 's')
        # Escribir Generador
        generador.setFrequency(freq)
        # Esperamos a que se setee y lea bien
        time.sleep(1)
        # Consulta Osciloscopio
        values[i] = osci.vpp()
    return freqs, values

In [10]:
rm = visa.ResourceManager()
ID=rm.list_resources()
genf = Generador(ID[0])
osci = Osciloscopio(ID[1])

In [11]:
with osci as osciloscopio, genf as generador:
    frecuencias, valores = sweepe(generador, osciloscopio, cant_med = 20)
    plt.plot(frecuencias,valores)
    plt.show()


1.0


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

ID

In [7]:
with osci as osciloscopio, genf as generador:   
    osciloscopio.vertical_scale = 2.0

C:\Users\gonza\Anaconda3\lib\site-packages\lantz\core\processors.py:104: DimensionalityWarning: Assuming units `volt` for 2.0
  warnings.warn(msg, DimensionalityWarning)


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.